In [1]:
from string import ascii_lowercase

alphabet = ascii_lowercase

with open("/usr/share/dict/words") as f:
    words_all = f.read().splitlines()

# filter words with all lowercase letters, at least 4 letters, at most 7 distinct letters
f = lambda w: all(c in alphabet for c in w) and len(w) >= 4 and len(set(w)) <= 7

words = list(filter(f, words_all))

print(len(words_all), len(words))
print(words[:50])

104334 41351
['aardvark', 'aardvarks', 'abaci', 'aback', 'abacus', 'abacuses', 'abaft', 'abalone', 'abalones', 'abandon', 'abandoned', 'abandoning', 'abandons', 'abase', 'abased', 'abasement', 'abases', 'abash', 'abashed', 'abashes', 'abashing', 'abasing', 'abate', 'abated', 'abatement', 'abates', 'abating', 'abattoir', 'abattoirs', 'abbess', 'abbesses', 'abbey', 'abbeys', 'abbot', 'abbots', 'abbr', 'abbrev', 'abbreviate', 'abbrevs', 'abdicate', 'abdicated', 'abdomen', 'abduct', 'abducted', 'abductee', 'abducts', 'abeam', 'abed', 'aberrant', 'abet']


In [2]:
def word_score(word):
    # 4-letter word gets 1 point, longer words get length points
    # pangram gets 7 extra points
    if len(word) == 4: return 1
    else: return len(word) + 7 * (len(set(word)) == 7)

print(word_score("area"))
print(word_score("hello"))
print(word_score("inglenook"))

1
5
16


In [3]:
# for efficiency, make words into bitmasks of letters: every word is a 26-bit mask
# the candidate letterset is also a 26-bit mask

from collections import defaultdict, Counter

# for each mask, store words with that mask and total word score
mask_scores = Counter()
mask_words = defaultdict(list)

for word in words:
    mask = 0
    for c in word:
        mask |= 1 << (ord(c) - ord('a'))

    mask_scores[mask] += word_score(word)
    mask_words[mask].append(word)

print(len(mask_scores))

21018


In [4]:
# center specified as index 0-6, -1 means ignore
def mask_score(mask, center, return_words=False):
    # indices of the mask bits
    inds = [i for i in range(26) if mask & (1 << i)]

    score = 0
    words = []
    # go through all 7-bit combos except empty
    for i in range(1, 1 << 7):  
        if center >= 0 and not i & (1 << center):  # require center
            continue
        subset_mask = 0
        for j in range(7):
            if i & (1 << j):
                # set subset mask bits
                subset_mask |= 1 << inds[j] 

        score += mask_scores[subset_mask]
        if return_words:
            words += mask_words[subset_mask]

    if return_words:
        return words
    return score

best_score = 0
best_words = None 

# brute force (26 choose 7) masks
for mask in range(1 << 26):
    if mask.bit_count() != 7:
        continue

    possible_score = mask_score(mask, -1)
    # don't bother if max possible score is less than best score so far
    if possible_score <= best_score:
        continue

    for center in range(7):
        score = mask_score(mask, center)
    
        if score > best_score:
            best_score = score
            best_words = mask_score(mask, center, return_words=True)
            letters = [alphabet[i] for i in range(26) if mask & (1 << i)]
            letters[center] = letters[center].upper()
            print(score, "".join(letters))

200 Abcdefg
220 abcDefg
250 abcdEfg
271 abcdEgh
284 abcDefi
286 abcdEfi
292 acdEfhi
347 Abcdefl
410 abcdEfl
462 abcdEgl
490 abdEfgl
541 abcdEil
594 abdEfil
664 abdEgil
690 abdEilm
749 abdEiln
766 abeGiln
774 abegIln
791 Adegiln
838 aDegiln
938 adEgiln
960 adegilN
1019 abdEgir
1056 Abdeglr
1219 abdEglr
1226 abdEilr
1410 Adeginr
1544 aDeginr
1811 adEginr
1962 adEilrs
2044 adEgnrs
2203 dEginrs
2542 adEiprs
2643 acdErst
2658 acEhrst
2725 adEhrst
3212 adEirst
3408 aEinrst


In [5]:
# show word set, capitalize pangrams
print(" ".join(w.upper() if len(set(w)) == 7 else w for w in best_words))

nine inane area rare rarer rear eerie eerier aerie airier arena earn earner near nearer reran inner rein inaner rainier sees asses assess assesses ease eases sasses seas sises sissies seen sense senses sane senna nines ninnies sine anise asinine easiness insane nannies errs seer seers sere serer areas ares arrears ears eras erase eraser erasers erases rares rears reassess reassesses saree sarees sear sears sera irises reis rise riser risers rises series sire sires sissier aeries arise arises easier raise raises sassier sierra sierras serene sereneness serener sneer sneers arenas earners earns ensnare ensnares nearness nears rareness saner snare snares eeriness reins resin resins rinse rinses risen sinner sinners siren sirens airiness arisen insaner teat entente teen tenet tent ante antenna antennae eaten neat tenant intent nineteen nite tine initiate innate teeter tree aerate eater errata rate retreat tare tarter tatter tear treat retire retiree rite terrier tier tire titter trite trit